## Task 3: Correlation between news and stock Movement

In [1]:
# reload the module
%load_ext autoreload
%autoreload 2 

In [2]:
# import libraries
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import cm
import pandas as pd
import os


In [3]:
import sys
from pathlib import Path

# Add the project root directory to sys.path
project_root = Path("..").resolve()
if str(project_root) not in sys.path:
    sys.path.append(str(project_root))

In [4]:
#import data loader modules
from src.data_loader import DataLoader as news_dt
from src.y_finance_data_loader import DataLoader as y_finance_dt

### Load both datasets

In [5]:
#load news data
news_data_loader = news_dt("../data/raw_analyst_ratings/raw_analyst_ratings.csv");
news_df = news_data_loader.load_data()
print(news_df.head())

# Convert news_df date to timezone-aware datetime
if news_df['date'].dt.tz is None:
    news_df['date'] = pd.to_datetime(news_df['date']).dt.tz_localize('UTC')
else:
    news_df['date'] = pd.to_datetime(news_df['date']).dt.tz_convert('UTC')

#load y finance data
directory = '../data/yfinance_data'
aligned_dfs = []
# List all files in the directory
file_names = os.listdir(directory)

for file_name in file_names:
    if os.path.isfile(os.path.join(directory, file_name)):
        print(os.path.join(directory, file_name))
            # Load the stock data
        stock_data = pd.read_csv(os.path.join(directory, file_name))
        
        # Convert the 'Date' column to datetime with timezone UTC
        stock_data['Date'] = pd.to_datetime(stock_data['Date']).dt.tz_localize('UTC')
        
        # Filter stock data to start from the first date in the news data
        start_date = news_df['date'].min()
        filtered_stock_data = stock_data[stock_data['Date'] >= start_date]
        
        # Perform an inner join to keep only matching dates
        aligned_data = pd.merge(filtered_stock_data, news_df, left_on='Date', right_on='date', how='inner')
        
        # Add a column to identify the stock
        stock_name = os.path.basename(file_name).split('_')[0]  # Extract stock name from file name
        aligned_data['Stock'] = stock_name
        
        # Append the aligned data to the list
        aligned_dfs.append(aligned_data)

# Concatenate all aligned data into one DataFrame
final_data = pd.concat(aligned_dfs, ignore_index=True)
        

   Unnamed: 0                                           headline  \
0           0            Stocks That Hit 52-Week Highs On Friday   
1           1         Stocks That Hit 52-Week Highs On Wednesday   
2           2                      71 Biggest Movers From Friday   
3           3       46 Stocks Moving In Friday's Mid-Day Session   
4           4  B of A Securities Maintains Neutral on Agilent...   

                                                 url          publisher  \
0  https://www.benzinga.com/news/20/06/16190091/s...  Benzinga Insights   
1  https://www.benzinga.com/news/20/06/16170189/s...  Benzinga Insights   
2  https://www.benzinga.com/news/20/05/16103463/7...         Lisa Levin   
3  https://www.benzinga.com/news/20/05/16095921/4...         Lisa Levin   
4  https://www.benzinga.com/news/20/05/16095304/b...         Vick Meyer   

                       date stock  
0 2020-06-05 10:30:54-04:00     A  
1 2020-06-03 10:45:20-04:00     A  
2 2020-05-26 04:30:07-04:00     

In [ ]:
final_data